<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:


      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-17 09:21:20
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.42 C
-------------------
Today PnL: 76.65 K (0.54%)
Current PnL: -16.77 L (-11.28%)
CY Booked + Current PnL: -5.74 L (-3.86%)
-------------------
Total profit:  2.32 L
Total loss:  -19.09 L
-------------------
Total Booked + Current PnL: 21.24 L (17.46%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.74%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.66 L (58.04%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.76%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SBIN,863.00,-11.34,67.0,M-LC,4.26,220467.0,18988.0,8223.0,0.87,9.42,3.73,13.51,60.0,2.31,1.57,23.21,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.20,59.0,H-LC,11.07,173481.0,17304.0,8795.0,0.88,11.08,5.07,16.71,22.0,1.97,1.24,30.45,X40N,BTT,HEALTHCARE
78,TTKPRESTIG,770.00,105.98,62.0,M-SC,4.41,88720.0,-12057.0,12155.0,0.76,-11.96,13.70,0.09,245.0,-0.99,0.63,16.56,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.00,71.0,H-MC,5.73,115302.0,14798.0,15842.0,0.35,14.72,13.74,30.49,79.0,0.93,0.82,51.90,MH,ATH,METALS
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202299.0,2161.0,18733.0,0.25,1.08,9.26,10.44,4.0,0.12,1.44,6.70,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-20.77,44.0,H-MC,2.22,136289.0,501.0,42250.0,-0.36,0.37,31.00,31.48,107.0,0.01,0.97,12.53,X40,ATH,INSURANCE
12,BANKINDIA,190.00,-29.19,63.0,H-MC,5.83,181469.0,1661.0,110751.0,0.00,0.92,61.03,62.52,88.0,0.01,1.29,35.33,XR,NTT,BANKS
52,PGHH,17973.08,-30.94,51.0,H-MC,4.38,199830.0,-990.0,69761.0,0.55,-0.49,34.91,34.25,80.0,-0.01,1.42,4.68,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-17.63,35.0,H-LC,5.63,296606.0,-19236.0,131901.0,-1.43,-6.09,44.47,35.67,5.0,-0.15,2.11,6.92,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-13.32,45.0,H-LC,8.16,218812.0,-32956.0,86168.0,-0.64,-13.09,39.38,21.13,27.0,-0.38,1.56,17.70,X40,ATH,PAINTS
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.12,230240.0,-11684.0,74943.0,-0.05,-4.83,32.55,26.15,8.0,-0.16,1.64,8.80,X40,ATH,IT
20,CIPLA,1795.00,-20.12,50.0,H-LC,5.81,213309.0,8809.0,32615.0,0.56,4.31,15.29,20.25,10.0,0.27,1.52,13.42,X40N,BTT,PHARMA
44,JIOFIN,387.00,0.31,52.0,H-LC,13.40,272620.0,5255.0,60194.0,0.52,1.97,22.08,24.48,48.0,0.09,1.94,58.02,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.20,-2.31,63.0,H-LC,1.01,242325.0,-3826.0,83045.0,0.81,-1.55,34.27,32.18,16.0,-0.05,1.73,33.93,X200,ATH,IT
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202299.0,2161.0,18733.0,0.25,1.08,9.26,10.44,4.0,0.12,1.44,6.70,X40,NTT,FMCG
51,NESTLEIND,1377.00,-8.22,58.0,H-LC,4.14,282344.0,16918.0,39867.0,-0.49,6.37,14.12,21.40,11.0,0.42,2.01,13.92,XY25,NTT,FMCG
57,RELIANCE,1533.00,-12.16,59.0,H-LC,5.60,220288.0,9922.0,18857.0,0.91,4.72,8.56,13.68,37.0,0.53,1.57,22.02,XY25,NTT,REFINERIES
84,VBL,671.64,-17.63,35.0,H-LC,5.63,296606.0,-19236.0,131901.0,-1.43,-6.09,44.47,35.67,5.0,-0.15,2.11,6.92,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,63.09,69.0,H-SC,8.54,141316.0,-41459.0,152522.0,5.28,-22.68,107.93,60.77,125.0,-0.27,1.01,26.57,XR,NTT,JEWELLERY
55,RAJOOENG,143.33,-33.95,45.0,H-SC,10.62,100150.0,-2350.0,43185.0,3.03,-2.29,43.12,39.83,114.0,-0.05,0.71,4.68,AR,ATH,MISC
77,TRIDENT,48.00,-5.29,64.0,M-SC,5.35,73705.0,-18550.0,44857.0,2.90,-20.11,60.86,28.51,224.0,-0.41,0.53,26.12,XR,NTT,TEXTILES
56,RELAXO,1176.00,-32.04,57.0,H-SC,1.76,92646.0,-52514.0,128444.0,2.46,-36.18,138.64,52.31,136.0,-0.41,0.66,24.01,X40N,NTT,FOOTWEAR
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,41.94,87360.0,-26189.0,66236.0,2.43,-23.06,75.82,35.27,268.0,-0.40,0.62,113.82,XR,NTT,HOTELS


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SFL,1287.00,39.06,49.0,M-SC,8.40,183554.0,-78664.0,148495.0,-2.36,-30.00,80.90,26.63,229.0,-0.53,1.31,14.12,XY24,NTT,MISC
79,UJJIVANSFB,60.00,96.03,56.0,H-SC,12.42,123903.0,-18576.0,38100.0,-1.78,-13.04,30.75,13.70,163.0,-0.49,0.88,46.24,OX40N,NTT,BANKS
65,SHALBY,327.00,1065.87,65.0,M-SC,2.56,158628.0,-22515.0,67988.0,-1.65,-12.43,42.86,25.10,235.0,-0.33,1.13,27.12,XY24,NTT,HEALTHCARE
37,INDIAMART,4850.92,-51.28,48.0,H-SC,12.14,136236.0,12900.0,120869.0,-1.48,10.46,88.72,108.45,119.0,0.11,0.97,35.15,AR,ATH,MISC
84,VBL,671.64,-17.63,35.0,H-LC,5.63,296606.0,-19236.0,131901.0,-1.43,-6.09,44.47,35.67,5.0,-0.15,2.11,6.92,X40N,ATH,FMCG


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,105.98,62.0,M-SC,4.41,88720.0,-12057.0,12155.0,0.76,-11.96,13.70,0.09,245.0,-0.99,0.63,16.56,OX40N,NTT,DURABLES
79,UJJIVANSFB,60.0,96.03,56.0,H-SC,12.42,123903.0,-18576.0,38100.0,-1.78,-13.04,30.75,13.70,163.0,-0.49,0.88,46.24,OX40N,NTT,BANKS
19,CERA,9475.0,-19.81,48.0,H-SC,2.15,146280.0,-29623.0,71648.0,0.65,-16.84,48.98,23.89,149.0,-0.41,1.04,26.33,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-19.51,54.0,H-MC,7.78,108794.0,-23041.0,65266.0,0.72,-17.48,59.99,32.03,98.0,-0.35,0.78,21.86,OX40N,NTT,IT
45,KANSAINER,340.0,-67.79,57.0,H-SC,1.76,221958.0,-47709.0,84033.0,0.19,-17.69,37.86,13.47,138.0,-0.57,1.58,12.26,XY24,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-25.47,57.0,H-LC,12.79,296476.0,-49480.0,124342.0,1.35,-14.30,41.94,21.64,1.0,-0.40,2.11,5.03,X40,ATH,IT
40,INFY,2275.00,-17.09,54.0,H-LC,8.78,322929.0,10041.0,161658.0,0.51,3.21,50.06,54.87,3.0,0.06,2.30,10.07,X40,BTT,IT
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202299.0,2161.0,18733.0,0.25,1.08,9.26,10.44,4.0,0.12,1.44,6.70,X40,NTT,FMCG
84,VBL,671.64,-17.63,35.0,H-LC,5.63,296606.0,-19236.0,131901.0,-1.43,-6.09,44.47,35.67,5.0,-0.15,2.11,6.92,X40N,ATH,FMCG
1,ABB,7934.00,-38.31,64.0,H-LC,11.11,257784.0,-3835.0,123040.0,0.59,-1.47,47.73,45.57,7.0,-0.03,1.84,9.69,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.95,66.0,M-MC,6.98,230042.0,5080.0,162571.0,0.60,2.26,70.67,74.53,192.0,0.03,1.64,35.24,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,11.62,59.0,H-MC,3.70,192580.0,27910.0,40057.0,1.94,16.95,20.80,41.28,89.0,0.70,1.37,26.28,X40N,ATH,AC
30,HAVELLS,2069.16,1.47,66.0,H-MC,1.83,251675.0,3944.0,71123.0,1.13,1.59,28.26,30.30,92.0,0.06,1.79,15.10,X40,ATH,ELECTRICAL
85,VOLTAS,1918.49,1.30,59.0,H-MC,2.68,212430.0,20688.0,75349.0,0.03,10.79,35.47,50.08,99.0,0.27,1.51,18.32,XY25,ATH,AC
24,DABUR,735.00,-3.66,55.0,H-MC,6.55,209781.0,12753.0,76864.0,-0.55,6.47,36.64,45.49,102.0,0.17,1.50,20.36,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.00,71.0,H-MC,5.73,115302.0,14798.0,15842.0,0.35,14.72,13.74,30.49,79.0,0.93,0.82,51.90,MH,ATH,METALS
86,WHIRLPOOL,2270.00,-40.41,54.0,M-SC,1.07,100770.0,9272.0,69481.0,-0.58,10.13,68.95,86.07,223.0,0.13,0.72,46.34,XR,NTT,DURABLES
37,INDIAMART,4850.92,-51.28,48.0,H-SC,12.14,136236.0,12900.0,120869.0,-1.48,10.46,88.72,108.45,119.0,0.11,0.97,35.15,AR,ATH,MISC
87,WIPRO,420.00,-10.16,57.0,M-LC,5.55,157678.0,6733.0,102727.0,1.21,4.46,65.15,72.51,53.0,0.07,1.12,10.75,XR,NTT,IT
36,IEX,219.00,-29.00,58.0,H-SC,17.27,202872.0,2918.0,94315.0,1.56,1.46,46.49,48.63,115.0,0.03,1.45,13.18,XR,NTT,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,41.94,87360.0,-26189.0,66236.0,2.43,-23.06,75.82,35.27,268.0,-0.40,0.62,113.82,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.34,48.0,H-SC,13.34,89074.0,-11889.0,106417.0,0.49,-11.78,119.47,93.63,148.0,-0.11,0.64,32.67,SR,ATH,CHEMICALS
49,MASFIN,398.61,-15.54,58.0,H-SC,12.92,95805.0,-2175.0,23779.0,-0.17,-2.22,24.82,22.05,152.0,-0.09,0.68,39.26,XR,ATH,FINANCE
50,NATIONALUM,247.44,-42.00,71.0,H-MC,5.73,115302.0,14798.0,15842.0,0.35,14.72,13.74,30.49,79.0,0.93,0.82,51.90,MH,ATH,METALS
46,KPIGREEN,731.05,3.26,45.0,H-SC,10.25,123102.0,-2195.0,61120.0,1.65,-1.75,49.65,47.03,141.0,-0.04,0.88,54.13,MH,ATH,POWER


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3894.59,65.0,M-SC,6.46,124549.0,-22819.0,74630.0,1.91,-15.48,59.92,35.16,236.0,-0.31,0.89,27.21,XY24,NTT,AUTO
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,41.94,87360.0,-26189.0,66236.0,2.43,-23.06,75.82,35.27,268.0,-0.40,0.62,113.82,XR,NTT,HOTELS
50,NATIONALUM,247.44,-42.00,71.0,H-MC,5.73,115302.0,14798.0,15842.0,0.35,14.72,13.74,30.49,79.0,0.93,0.82,51.90,MH,ATH,METALS
61,SAMMAANCAP,326.00,-171.59,63.0,M-SC,2.62,82020.0,-20190.0,113581.0,0.66,-19.75,138.48,91.37,208.0,-0.18,0.58,33.15,XY25,NTT,FINANCE
76,TITAGARH,1548.00,0.74,69.0,H-SC,4.11,165078.0,-26937.0,105815.0,-0.53,-14.03,64.10,41.08,158.0,-0.25,1.18,39.88,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.47
1,25,42.71
2,50,73.14


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.29
LC    33.95
MC    22.74
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.51
X40      14.98
X40N     12.77
XR       11.42
XY25      9.61
AR        8.51
OX40N     8.05
X200      1.73
MH        1.70
X5K       1.43
SR        1.27
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.44
H-LC    25.82
H-MC    19.82
M-SC    13.31
M-LC     7.07
M-MC     2.62
L-SC     1.54
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.17,-13.87,75.95
FMCG,11.57,-4.24,40.71
FINANCE,9.51,-15.29,60.45
BANKS,7.62,-12.01,63.57
MISC,7.34,-11.55,73.59
PAINTS,5.72,-15.13,32.29
ELECTRICAL,5.60,-4.68,62.37
AC,3.58,3.70,28.91
INSURANCE,3.36,-6.71,43.32


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2846162.0,22
XR,1309102.0,14
AR,1302585.0,9
X40,744039.0,10
X40N,621043.0,10
OX40N,519085.0,10
XY25,402403.0,7
SR,244607.0,2
X5K,117147.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3426112.0,29
M-SC,1378235.0,17
H-LC,1250922.0,15
H-MC,1171831.0,15
M-LC,386054.0,5
M-MC,310774.0,2
L-SC,245690.0,3
L-MC,60317.0,1
L-LC,36245.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1033106.0      6
M-SC       XY24       798608.0      7
H-SC       AR         785744.0      5
           XR         758325.0      7
H-MC       XY24       485523.0      4
H-LC       X40        465844.0      5
           AR         355629.0      2
M-MC       XY24       310774.0      2
H-SC       X40N       294483.0      4
           OX40N      248727.0      4
           SR         244607.0      2
H-LC       X40N       227486.0      4
H-MC       X40        209745.0      4
           XY25       185630.0      2
L-SC       XR         162644.0      2
M-SC       AR         161212.0      2
M-LC       XY24       157957.0      2
M-SC       OX40N      122046.0      4
M-LC       X5K        117147.0      1
M-SC       XR         114338.0      2
           XY25       113581.0      1
H-MC       XR         110751.0      1
M-LC       XR         102727.0      1
H-MC       X40N        99074.0      2
L-SC       OX40N       83046.0      1
H-LC       X200        83045.0      1
M-SC       X40         68450.0      1
H-MC       OX40N       65266.0      1
H-SC       MH          61120.0      1
L-MC       XR          60317.0      1
H-LC       XY24        60194.0      1
           XY25        58724.0      2
L-LC       XY25        36245.0      1
H-MC       MH          15842.0      1
M-LC       XY25         8223.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
